# Ovde ide dobijeni beat

In [1]:
import random

In [32]:
tempo = 90

In [3]:
def calculate_weights(string):  
    weights = []
    current_weight = 1/8

    index = 1
    while index < len(string):
        if string[index] == '1':
            weights.append(current_weight)
            current_weight = 1/8
        elif string[index] == '0':
            current_weight += 1/8
        index += 1
    if index==len(string):
            weights.append(current_weight)
    
    return weights

In [4]:
def calculate_weights_from_file(filename):
    all_weights = []

    with open("beats/"+filename+".txt", 'r') as file:
        for line in file:
            string = line.strip() 
            weights = calculate_weights(string)
            all_weights.append(weights)

    return all_weights


In [5]:
filename = input("Beats file name (without extension): ")
beats=calculate_weights_from_file(filename)
print(beats)

Beats file name (without extension):  primer1


[[0.375, 0.125, 0.125, 0.125, 0.25, 0.375, 0.25, 0.25, 0.125], [0.125, 0.125, 0.25, 0.125, 0.125, 0.25, 0.25, 0.25, 0.125, 0.125, 0.125, 0.125]]


In [6]:
# Primer beat-a
#beats = [
#    [1/4, 1/8, 1/8, 1/4, 1/8, 1/8, 1/4, 1/8, 1/8, 1/2],
#    [1/4, 1/8, 1/8, 1/4, 1/4, 3/8, 1/8, 1/2],
#   [1/2, 1/2, 1/4, 1/8, 1/8, 1/4, 1/8, 3/8]
#]
# Ovde ce biti sacuvana izabrana lestvica
scale = []

# Potrebna muzicka teorija

In [7]:
def note_duration(note_length):
    bar_duration = 60 / tempo * 4
    return note_length * bar_duration

In [8]:
# Keys
#              0    1     2    3     4    5     6    7    8     9    10    11
piano_keys = ['c', 'cs', 'd', 'ds', 'e', 'f', 'fs', 'g', 'gs', 'a', 'as', 'b']

# Modes
modes = {
    "major" : [0, 2, 4, 5, 7, 9, 11],
    "minor" : [0, 2, 3, 5, 7, 8, 10]
}

In [9]:
# Note u skali
def scale(root, mode):
    scl = []
    n = piano_keys.index(root)

    for i in modes[mode]:
        scl.append(piano_keys[(i+n)%12])

    return scl

In [10]:
for mode in modes:
    for key in piano_keys:
        print(key + " " + mode + ": ", scale(key, mode))

c major:  ['c', 'd', 'e', 'f', 'g', 'a', 'b']
cs major:  ['cs', 'ds', 'f', 'fs', 'gs', 'as', 'c']
d major:  ['d', 'e', 'fs', 'g', 'a', 'b', 'cs']
ds major:  ['ds', 'f', 'g', 'gs', 'as', 'c', 'd']
e major:  ['e', 'fs', 'gs', 'a', 'b', 'cs', 'ds']
f major:  ['f', 'g', 'a', 'as', 'c', 'd', 'e']
fs major:  ['fs', 'gs', 'as', 'b', 'cs', 'ds', 'f']
g major:  ['g', 'a', 'b', 'c', 'd', 'e', 'fs']
gs major:  ['gs', 'as', 'c', 'cs', 'ds', 'f', 'g']
a major:  ['a', 'b', 'cs', 'd', 'e', 'fs', 'gs']
as major:  ['as', 'c', 'd', 'ds', 'f', 'g', 'a']
b major:  ['b', 'cs', 'ds', 'e', 'fs', 'gs', 'as']
c minor:  ['c', 'd', 'ds', 'f', 'g', 'gs', 'as']
cs minor:  ['cs', 'ds', 'e', 'fs', 'gs', 'a', 'b']
d minor:  ['d', 'e', 'f', 'g', 'a', 'as', 'c']
ds minor:  ['ds', 'f', 'fs', 'gs', 'as', 'b', 'cs']
e minor:  ['e', 'fs', 'g', 'a', 'b', 'c', 'd']
f minor:  ['f', 'g', 'gs', 'as', 'c', 'cs', 'ds']
fs minor:  ['fs', 'gs', 'a', 'b', 'cs', 'd', 'e']
g minor:  ['g', 'a', 'as', 'c', 'd', 'ds', 'f']
gs minor:  ['g

# Sviranje dobijene melodije

In [11]:
import pygame as pg
import time
from threading import Thread

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [12]:
pg.mixer.init()
pg.init()

(5, 0)

In [13]:
def path(note):
    return "Piano/" + note + ".wav"

In [14]:
def play_note(note, duration):
    pg.mixer.music.load(path(note))
    pg.mixer.music.play()
    time.sleep(note_duration(duration))

# Klasa Melody

In [15]:
import random

In [16]:
class Melody:

    def __init__(self, beats):
        self.notes = self.generate_melody(random.choice(beats))
        self.fitness = self.calc_fitness()

    def generate_melody(self, beat):
        notes = []
        for i in range(len(beat)):
            octave = random.randint(4,5)
            note = note = random.choice(scale)
            note = note + str(octave)
            notes.append([note, beat[i]])
        return notes

    def print(self):
        for note in self.notes:
            print(note[0], end=" ")
        print()

    def play(self):
        print("Melody", end=" ")
        self.print()
        pg.mixer.set_num_channels(len(self.notes))
        th = {}
        n = 0
        for i in self.notes:
            th[n] = Thread(target=play_note, args=(i[0], i[1]))
            th[n].start()
            th[n].join()
            n += 1

    def calc_fitness(self):
        self.play()
        rating = int(input("Rating: "))
        print()
        return rating

# Genetic Algorithm

In [17]:
def selection(population, tournament_size):
    chosen = random.sample(population, tournament_size)
    return max(chosen, key=lambda x: x.fitness)

In [18]:
def crossover(parent1, parent2, child1, child2):
    
    random_pos = random.randrange(0, len(parent1.notes))
    
    child1.notes[:random_pos] = parent1.notes[:random_pos]
    child1.notes[random_pos:] = parent2.notes[random_pos:]
    
    child2.notes[:random_pos] = parent2.notes[:random_pos]
    child2.notes[random_pos:] = parent1.notes[random_pos:]

In [19]:
def mutation(melody, mutation_prob):
    for i in range(len(melody.notes)):
        prob = random.random()
        if prob < mutation_prob:
            new_note = random.choice(scale)
            octave = random.randint(4,5)
            melody.notes[i][0] = new_note+str(octave)

In [20]:
def genetic_algorithm(
    num_bars=8, 
    scale_root="c", 
    scale_mode="major", 
    population_size=10,
    elitism_size=2, 
    tournament_size=2, 
    mutation_probability=0.5
):

    genId = 1
    print("GENERATION " + str(genId))
    print()
    population = [Melody(beats) for _ in range(population_size)]
    new_population = population.copy()

    while True:

        if input("Next generation? [y/n]: ") == 'n':
            break

        genId += 1
        print("GENERATION " + str(genId))
        print()
        
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = population[:elitism_size]
        
        for j in range(elitism_size, population_size, 2):
            
            parent1 = selection(population, tournament_size)
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, child1=new_population[j], child2=new_population[j+1])
    
            mutation(new_population[j], mutation_probability)
            mutation(new_population[j+1], mutation_probability)
            
            new_population[j].fitness = new_population[j].calc_fitness()
            new_population[j+1].fitness = new_population[j+1].calc_fitness()
    
        population = new_population.copy()
        
    population.sort(key=lambda x: x.fitness, reverse=True)
    return population[:8]

# Main

In [21]:
def save_melody(fileName, melodies):
    with open("melodies/" + fileName + ".txt", 'w') as f:
        for melody in melodies:
            for note in melody.notes:
                f.write(note[0] + " " + str(note[1]) + " ")
            f.write("\n")

In [22]:
scale_root = input("Scale root [ex. c for C, cs for C#]: ")
scale_mode = input("Scale mode [ex. major, minor]: ")
scale = scale(scale_root, scale_mode)

Scale root [ex. c for C, cs for C#]:  c
Scale mode [ex. major, minor]:  major


In [23]:
melodies = genetic_algorithm()

GENERATION 1

Melody e5 b5 e5 f5 d5 a4 f4 d5 a5 


Rating:  5



Melody g5 g5 d5 d5 e4 b5 g5 d5 g4 


Rating:  5



Melody c4 d4 e4 d5 f5 a5 b5 f5 d5 d5 g5 d5 


...


Melody g5 d5 b5 f4 e5 a4 a5 c5 c5 


Rating:  5



Melody f4 e5 c5 b4 d5 e5 c4 e4 b5 e4 f5 g4 


Rating:  5


Next generation? [y/n]:  n


In [24]:
for melody in melodies:
    melody.play()

Melody e5 b5 e5 f5 d5 a4 f4 d5 a5 
Melody g5 g5 d5 d5 e4 b5 g5 d5 g4 
Melody b5 b4 g4 a5 f4 c4 a5 g4 a4 
Melody g5 d5 b5 f4 e5 a4 a5 c5 c5 
Melody f4 e5 c5 b4 d5 e5 c4 e4 b5 e4 f5 g4 
Melody f4 d5 e5 d4 b5 b4 e4 g5 d5 
Melody c4 d5 e5 e5 d4 e5 a5 b5 a4 
Melody g5 b4 b5 f5 f4 g4 b4 g5 c5 


In [25]:
ans = input("Do you want to save this melody? [y/n]: ")
if ans == "y":
    fileName = input("File name (without the extension): ")
    save_melody(fileName, melodies)

Do you want to save this melody? [y/n]:  y
File name (without the extension):  primer3


# Loading saved melodies

In [26]:
def play(melody):
    pg.mixer.set_num_channels(len(melody))
    th = {}
    n = 0
    for i in melody:
        th[n] = Thread(target=play_note, args=(i[0], i[1]))
        th[n].start()
        th[n].join()
        n += 1

In [27]:
def load_melody(filename):
    result = []
    with open("melodies/" + filename + ".txt", "r") as f:
        for line in f:
            line_melody = []
            words = line.split()
            n = 0
            while n < len(words):
                line_melody.append([words[n], float(words[n+1])])
                n += 2
            result.append(line_melody)

    return result

In [28]:
filename = input("Melody file name (without extension): ")
loaded_melody = load_melody(filename)

Melody file name (without extension):  primer3


In [33]:
for melody in loaded_melody:
    play(melody)